In [ ]:
# 설치 파일
# %pip install ipykernel
# %pip install jupyter
# %pip install pandas

# %pip install nltk

# %pip install konlpy

# %pip install tensorflow
# %pip install keras 

# 띄어쓰기 교정 - 실행 Error
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git

# kss - 문장 분리
# %pip install kss

# soynlp - Normalizer(정규화)
# %pip install soynlp

# 맞춤법 교정 - 실행 Error
# !pip install git+https://github.com/ssut/py-hanspell.git
# %pip install py-hanspell 

# 품사 태깅
# %pip install pykomoran

In [ ]:
# mecab 테스트
from konlpy.tag import Mecab

Mecab('C:\mecab\mecab-ko-dic')


### 전처리
1. 정규화
2. 문장 토큰화
3. 단어 토큰화
4. 자연어처리기 활용 토큰화 -> 품사태깅
   - NNP : 고유명사
   - NNG : 일반명사
   - VA : 형용사
   - VV : 동사
5. 불용품사, 불용어 제거
6. 표제어 추출
   
### Word Representation
1. One-Hot Encoding
2. Label Encoding -> 단어 -> Indexing
Index -> 단어 뽑기
1. TDM
문서 별 단어 빈도 카운팅
행 = 단어 / 열 = 문서
1. TF-IDF
만든 TDM을 가지고,
문서 별 TF 계산 / DF 계산 / -> TF-IDF 계산  


##### 분석대상 텍스트

In [ ]:
import pandas as pd

file_name = 'edaily.csv'

edaily = pd.read_csv(file_name, sep=',')

# len(edaily)
# edaily

# text_list = edaily['ARTICLE']
# text_list

# Test 10개
text_list = edaily['ARTICLE']

edaily_list = [] 
for i in range(1, 10):
    edaily_list.append(text_list[i])

# edaily_list = edaily['ARTICLE']    
edaily_list

##### 텍스트 정제(cleaning)

In [ ]:
# punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
# punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

# def clean_punc(text, punct, mapping):
#     for p in mapping:
#         text = text.replace(p, mapping[p])

#     for p in punct:
#         text = text.replace(p, f' {p} ')

#     specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
#     for s in specials:
#         text = text.replace(s, specials[s])

#     return text.strip()

# corpus = list(map(lambda x : clean_punc(x, punct, punct_mapping), corpus))

In [ ]:
# 텍스트 정제 함수

import re

def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9]'    # 숫자와 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    
    # '이데일리' 텍스트 제거
    pattern = '이데일리' 
    text = re.sub(pattern, '', text)

    return text

In [ ]:
# 텍스트 정제 및 문장 토큰화
from soynlp.normalizer import *

pattern = '\d+.\d+' # 소수점

article = []
for text in edaily_list:
    text = repeat_normalize(text, num_repeats=2)

    text = re.sub(pattern, '', text)   
    sent_list = text.replace('다.', '다||').split('||')
    
    for sent in sent_list:
        article.append(sent)        

clean_list = list(map(lambda x : text_clean(x), article))

clean_list

In [ ]:
# 텍스트 정제(문장 토큰화 제외)
corpus = [repeat_normalize(text, num_repeats=2) for text in edaily_list]
    
clean_list = list(map(lambda x : text_clean(x), corpus))

clean_list

In [ ]:
# 띄어쓰기 -> 띄어쓰기 보다 문장 분리 권장
from pykospacing import Spacing

spacing = Spacing()

spacing_list = list(map(lambda x : spacing(x), clean_list))

spacing_list

In [ ]:
# 문장 분리

import kss

# sentence_list = [kss.split_sentences(clean) for clean in clean_list]

sentence_list = list(map(lambda x : kss.split_sentences(x), clean_list))

# sentence_list
sent_list = []
for sentence in sentence_list:
    for sent in sentence:
        sent_list.append(sent)

sent_list

In [ ]:
# 맞춤법 교정 - Result Error
# from hanspell import spell_checker

# test = [
#     "김형호영화시장분석가는'1987'의네이버영화정보네티즌	10점평에서언급된단어들을지난해12월27일부터올해1월10일까지통계프로그램R과KoNLP패키지로텍스트마이닝하여분석했다.",
#     "김형호영화시장분석가는'1987'의네이버영화정보네티즌	10점평에서언급된단어들을지난해12월27일부터올해1월10일까지통계프로그램R과KoNLP패키지로텍스트마이닝하여분석했다."
#     ]

# result = [spell_checker.check(t) for t in test]
# result.as_dict()

# spell_list = list(map(lambda x : spell_checker.check(x).checked, test))

# spell_list

In [85]:
# 품사 태깅 및 불용어 처리
from konlpy.tag import Mecab

mecab = Mecab('C:\mecab\mecab-ko-dic')

# tokens = list(map(lambda x : mecab.morphs(x), sent_list))
# tokens = list(map(lambda x : mecab.pos(x), sent_list))

tokens = []
for sent in sent_list:
    for key, value in mecab.pos(sent):
        if value in ['NNG', 'VA', 'VAX','MAG','VA']:
            tokens.append(key)

print(tokens)    
# print("형태소:", tokens)


['기자', '다음', '일자', '신문', '주요', '기사', '다면', '기술', '유출', '수사', '팀', '격차', '베', '파트너', '반도체', '산업', '확대', '때', '철강', '배터리', '소재', '쌍두마차', '고령화', '노인', '복지', '강화', '분양', '실버타운', '사설', '자국', '산업', '보호', '경제', '질서', '판인', '텔', '보조금', '보라', '사설', '장년층', '일자리', '불안', '세계', '최악', '개혁', '과제', '다면', '프로', '야구', '개막', '귀환', '연패', '도전', '야구', '열기', '올해', '심상', '참', '다', '주심', '대신', '로봇', '심판', '스트라이크', '尹', '정부', '건강', '행복', '노후', '대책', '노인', '공공', '임대', '공급', '배', '확대', '중산', '도입', '하반기', '치매', '관리', '주치', '도입', '중증', '방문', '진료비', '절반', '정부', '이탈', '전공', '다음', '주', '면허', '정지', '제회', '국제', '비즈니스', '금융', '컨퍼런스', '글로벌', '금융', '산업', '격변기', '베', '협력', '매직', '인센티브', '디지털', '디지털', '전환', '은행', '기회', '韓', '기업', '최적', '제공', '참석자', '열기', '베', '금융', '산업', '관계자', '북적', '중점', '검찰청', '개발자', '출신', '검사', '특허', '자', '문관', '기술', '유출', '사건', '지난해', '해결', '대검', '총장', '응원군', '핫라인', '구축', '기소', '쑥', '면', '종합', '연내', '회', '금리', '인하', '유지', '비둘기', '파월', '시장', '환호성', '배터리', '소재', '시장', '안', '좋', '적극', '투자', '철강', '이미지', '단

In [86]:
f = open('stop_words.txt', 'r', encoding='utf=8')

stop_words = f.readlines()

stop_words = [word.replace('\n', '') for word in stop_words]

# stop_words

st_tokens = []
for token in tokens:
  if token not in stop_words:
    st_tokens.append(token)

tokens = st_tokens # 변수의 편의성을 위해 다시 tokens로 변수명 변경

tokens

['일자',
 '신문',
 '주요',
 '기사',
 '다면',
 '기술',
 '유출',
 '수사',
 '팀',
 '격차',
 '베',
 '파트너',
 '반도체',
 '산업',
 '확대',
 '철강',
 '배터리',
 '소재',
 '쌍두마차',
 '고령화',
 '노인',
 '복지',
 '강화',
 '분양',
 '실버타운',
 '사설',
 '자국',
 '산업',
 '보호',
 '경제',
 '질서',
 '판인',
 '텔',
 '보조금',
 '보라',
 '사설',
 '장년층',
 '일자리',
 '불안',
 '세계',
 '최악',
 '개혁',
 '과제',
 '다면',
 '프로',
 '야구',
 '개막',
 '귀환',
 '연패',
 '도전',
 '야구',
 '열기',
 '올해',
 '심상',
 '다',
 '주심',
 '대신',
 '로봇',
 '심판',
 '스트라이크',
 '尹',
 '정부',
 '건강',
 '행복',
 '노후',
 '대책',
 '노인',
 '공공',
 '임대',
 '공급',
 '배',
 '확대',
 '중산',
 '도입',
 '하반기',
 '치매',
 '관리',
 '주치',
 '도입',
 '중증',
 '방문',
 '진료비',
 '절반',
 '정부',
 '이탈',
 '전공',
 '주',
 '면허',
 '정지',
 '제회',
 '국제',
 '비즈니스',
 '금융',
 '컨퍼런스',
 '글로벌',
 '금융',
 '산업',
 '격변기',
 '베',
 '협력',
 '매직',
 '인센티브',
 '디지털',
 '디지털',
 '전환',
 '은행',
 '기회',
 '韓',
 '기업',
 '최적',
 '제공',
 '참석자',
 '열기',
 '베',
 '금융',
 '산업',
 '관계자',
 '북적',
 '중점',
 '검찰청',
 '개발자',
 '출신',
 '검사',
 '특허',
 '문관',
 '기술',
 '유출',
 '사건',
 '지난해',
 '해결',
 '대검',
 '총장',
 '응원군',
 '핫라인',
 '구축',
 '기소',
 '쑥',
 '면',
 '종합',
 '연내',

In [92]:
# 한 자 이상 글자 추출
tokens = [token for token in tokens if len(token) > 1]
# print(tokens[:50])
print(tokens)

['일자', '신문', '주요', '기사', '다면', '기술', '유출', '수사', '격차', '파트너', '반도체', '산업', '확대', '철강', '배터리', '소재', '쌍두마차', '고령화', '노인', '복지', '강화', '분양', '실버타운', '사설', '자국', '산업', '보호', '경제', '질서', '판인', '보조금', '보라', '사설', '장년층', '일자리', '불안', '세계', '최악', '개혁', '과제', '다면', '프로', '야구', '개막', '귀환', '연패', '도전', '야구', '열기', '올해', '심상', '주심', '대신', '로봇', '심판', '스트라이크', '정부', '건강', '행복', '노후', '대책', '노인', '공공', '임대', '공급', '확대', '중산', '도입', '하반기', '치매', '관리', '주치', '도입', '중증', '방문', '진료비', '절반', '정부', '이탈', '전공', '면허', '정지', '제회', '국제', '비즈니스', '금융', '컨퍼런스', '글로벌', '금융', '산업', '격변기', '협력', '매직', '인센티브', '디지털', '디지털', '전환', '은행', '기회', '기업', '최적', '제공', '참석자', '열기', '금융', '산업', '관계자', '북적', '중점', '검찰청', '개발자', '출신', '검사', '특허', '문관', '기술', '유출', '사건', '지난해', '해결', '대검', '총장', '응원군', '핫라인', '구축', '기소', '종합', '연내', '금리', '인하', '유지', '비둘기', '파월', '시장', '환호성', '배터리', '소재', '시장', '적극', '투자', '철강', '이미지', '단번에', '불식', '품질', '테스트', '통과', '승인', '인면', '정치', '보수', '결집', '호소', '호남', '민주', '연합', '지원', '귀국', '정치', '사퇴', 